# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [54]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger', 'maxent_ne_chunker'])
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /home/paperspace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponses.db')
df = pd.read_sql_table('Response', engine)

In [3]:
# check imported table
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
# specify dependent and independent variables
X = df.loc[:, ['message']]
y = df.iloc[:, 5:] # related is not a category

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # remove puntuaton and special chars
    text = re.sub(r'[^\w]', ' ', text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
        
    # lemmatize, normalize case, and remove leading/trailing white space
    stop_words = stopwords.words('english')
    final_tokens = [lemmatizer.lemmatize(token).strip().lower() 
                        for token in tokens 
                            if token not in stop_words and len(token) > 2]
    return final_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('count', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=42, n_jobs=-1))),
])   

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# create train test split
X_train, X_test, y_train, y_test = train_test_split(X.message.values, y, test_size=.25, random_state=42)

# train pipeline
pipeline.fit(X_train, y_train)

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/paperspace/anaconda3/l

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('count', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...           oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Make prediction
pred = pipeline.predict(X_test)

In [9]:
from sklearn.metrics import classification_report

# collect the classification reports
classification_reports = []
for index in range(len(y_test.columns)):
    classification_reports.append(classification_report(y_test.values[:, index], pred[:, index], output_dict=True))

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
# create dataframe for cleaner printing
results = pd.DataFrame(
    {'micro avg': [report['micro avg']['f1-score'] for report in  classification_reports],
     'macro avg': [report['macro avg']['f1-score'] for report in  classification_reports], 
     'weighted avg': [report['weighted avg']['f1-score'] for report in  classification_reports]}
)

In [11]:
# add total column
results = results.append(results.sum() / len(results), ignore_index=True)

In [12]:
# add category column
results = pd.concat([pd.DataFrame({'category': y_test.columns.append(pd.Index(['total'])).values}), results], axis=1, sort=False)

# print results
print(results)

                  category  micro avg  macro avg  weighted avg
0                  request   0.877769   0.736664      0.861571
1                    offer   0.995569   0.498890      0.993359
2              aid_related   0.744691   0.730095      0.740231
3             medical_help   0.917800   0.565796      0.889964
4         medical_products   0.951719   0.562845      0.933072
5        search_and_rescue   0.972956   0.616520      0.964653
6                 security   0.982888   0.495685      0.974557
7                 military   0.969442   0.561188      0.957543
8              child_alone   1.000000   1.000000      1.000000
9                    water   0.950191   0.712545      0.938770
10                    food   0.927731   0.758680      0.916591
11                 shelter   0.929717   0.705454      0.914952
12                clothing   0.986402   0.602766      0.981928
13                   money   0.979985   0.549361      0.971706
14          missing_people   0.989458   0.524747      0

In [13]:
print('Average f1-score over all categories: %s (weighted avg)' % results.iloc[-1, 3])

Average f1-score over all categories: 0.934744324399 (weighted avg)


In [14]:
pipeline.get_params()

{'count': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=<function tokenize at 0x7f9bcef0bbf8>, vocabulary=None),
 'count__analyzer': 'word',
 'count__binary': False,
 'count__decode_error': 'strict',
 'count__dtype': numpy.int64,
 'count__encoding': 'utf-8',
 'count__input': 'content',
 'count__lowercase': True,
 'count__max_df': 1.0,
 'count__max_features': None,
 'count__min_df': 1,
 'count__ngram_range': (1, 1),
 'count__preprocessor': None,
 'count__stop_words': None,
 'count__strip_accents': None,
 'count__token_pattern': '(?u)\\b\\w\\w+\\b',
 'count__tokenizer': <function __main__.tokenize>,
 'count__vocabulary': None,
 'memory': None,
 'multi_clf': MultiOutputClassifier(est

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
from sklearn.model_selection import GridSearchCV

# define hyperparameters of the diffferent estimators
parameters = {
        'count__tokenizer': [None, tokenize],
        #'count__stop_words': [None, 'english'],
        #'count__ngram_range': [(1,1), (1, 3)],
        'tfidf__use_idf' : [True, False],
        #'tfidf__norm': ['l2', 'l1'],
        'multi_clf__estimator__n_estimators': [20], #[10, 50, 100],
        #'multi_clf__estimator__oob_score' : [True, False],
        #'multi_clf__estimator__warm_start' : [True, False],
        'multi_clf__estimator__criterion' : ['gini', 'entropy']
}

# initialize grid search cross validation
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)

In [16]:
# fit model perform grid search
cv.fit(X_train, y_train)

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('count', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        stri...           oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'count__tokenizer': [None, <function tokenize at 0x7f9bcef0bbf8>], 'tfidf__use_idf': [True, False], 'multi_clf__estimator__n_estimators': [20], 'multi_clf__estimator__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
# Examine parameters, anything suprising?
cv.best_params_

{'count__tokenizer': None,
 'multi_clf__estimator__criterion': 'gini',
 'multi_clf__estimator__n_estimators': 20,
 'tfidf__use_idf': False}

In [19]:
# Make prediction with learned parameters
pred_grid = cv.predict(X_test)

In [67]:
def print_classification_summary(y_truth, y_predicted):
    # collect the classification reports
    reports = []
    for index in range(len(y_test.columns)):
        reports.append(classification_report(y_truth.values[:, index], y_predicted[:, index], output_dict=True))
        
    # create dataframe for cleaner printing
    results = pd.DataFrame(
        {'micro avg': [report['micro avg']['f1-score'] for report in  reports],
         'macro avg': [report['macro avg']['f1-score'] for report in  reports], 
         'weighted avg': [report['weighted avg']['f1-score'] for report in  reports]}
    )

    # add total column
    results = results.append(results.sum() / len(results), ignore_index=True)

    # add category column
    results = pd.concat([pd.DataFrame({'category': y_truth.columns.append(pd.Index(['total'])).values}), 
                              results], axis=1, sort=False)

    # print results
    print(results)
    print('Average f1-score over all categories: %s (weighted avg)' % results.iloc[-1, 3])

In [20]:
# collect the classification reports
classification_reports_grid = []
for index in range(len(y_test.columns)):
    classification_reports_grid.append(classification_report(y_test.values[:, index], pred_grid[:, index], output_dict=True))

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [63]:
# create dataframe for cleaner printing
results_grid = pd.DataFrame(
    {'micro avg': [report['micro avg']['f1-score'] for report in  classification_reports_grid],
     'macro avg': [report['macro avg']['f1-score'] for report in  classification_reports_grid], 
     'weighted avg': [report['weighted avg']['f1-score'] for report in  classification_reports_grid]}
)

# add total column
results_grid = results_grid.append(results_grid.sum() / len(results_grid), ignore_index=True)

# add category column
results_grid = pd.concat([pd.DataFrame({'category': y_test.columns.append(pd.Index(['total'])).values}), 
                          results_grid], axis=1, sort=False)

print('Average f1-score over all categories: %s (weighted avg)' % results_grid.iloc[-1, 3])

Average f1-score over all categories: 0.933793848606 (weighted avg)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [56]:
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) == 0:
                return False
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [57]:
class ResponseLengthExtractor(BaseEstimator, TransformerMixin):
    
    def fit(self, x, y=None):
        return self
    
    def response_length(self, text):
        return len(text)
    
    def transform(self, X):
        X_length = pd.Series(X).apply(self.response_length)
        return pd.DataFrame(X_length)

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

# First we try our starting verb extractor again but
# we combine our to clean up steps into one and add the length as separate feature
pipeline_extended = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('count', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('starting_verb', StartingVerbExtractor()),
        ('tweet_length', ResponseLengthExtractor()),
    ])),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=20))),
])

In [75]:
# fit model perform grid search
pipeline_extended.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, m...           oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=None))])

In [76]:
# Make prediction with learned parameters
pred_extended = pipeline_extended.predict(X_test)

In [77]:
print_classification_summary(y_test, pred_extended)

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                  category  micro avg  macro avg  weighted avg
0                  request   0.882353   0.742809      0.865565
1                    offer   0.995569   0.498890      0.993359
2              aid_related   0.761956   0.750763      0.759293
3             medical_help   0.917494   0.551112      0.887381
4         medical_products   0.950955   0.542793      0.930692
5        search_and_rescue   0.972345   0.551513      0.960597
6                 security   0.982582   0.495607      0.974404
7                 military   0.968678   0.536477      0.955617
8              child_alone   1.000000   1.000000      1.000000
9                    water   0.951413   0.719599      0.940273
10                    food   0.938579   0.817776      0.933773
11                 shelter   0.929106   0.699533      0.913617
12                clothing   0.986096   0.563163      0.980645
13                   money   0.979832   0.542846      0.971364
14          missing_people   0.989305   0.511200      0

### 9. Export your model as a pickle file

In [81]:
import pickle

# Save the model to disk
pickle.dump(pipeline_extended, open('models/nlp_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.